<a href="https://colab.research.google.com/github/LeomaxFilho/Operational-Research/blob/main/Pr%C3%A1tica_PLI_BinPacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula prática: Bin Packing


## Exercício 1


### Descrição do problema
Uma faculdade quer agendar as provas de 6 disciplinas. Existe uma regra que proíbe que o mesmo aluno tenha que fazer mais de uma prova por dia. A tabela abaixo mostra quais alunos (identificados por números) vão fazer a prova de cada disciplina. Por exemplo, A e B não podem ser agendadas no mesmo dia por causa do aluno 1. Já B e C poderiam ser agendadas no mesmo dia, pois nenhum aluno vai fazer essas duas provas.

| Disciplina | Alunos |
|:---:|:---:|
| A | 1, 2, 3, 4 |
| B | 1, 5 |
| C | 3, 6, 7 |
| D | 5, 7 |
| E | 2, 7 |
| F | 4, 5 |

Crie um modelo de PLI para agendar essas provas no menor número de dias possível.
**Dica: baseie-se no modelo do problema de empacotamento de caixas.**


### Resolução

In [ ]:
# instalação e importação do pacote mip
!pip install mip
from mip import *

# funcões usadas posteriormente:

# resolve o modelo e mostra os valores das variáveis
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  if status != OptimizationStatus.OPTIMAL:
    return

  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
    if v.x > 0.001:
      print(f"{v.name} = {v.x:.2f}")


# salva modelo em arquivo lp, e mostra o conteúdo
def save(model, filename):
  model.write(filename) # salva modelo em arquivo
  with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())

In [ ]:
model = # implementar modelo

save(model, "model.lp")

In [ ]:
solve(model)